In [1]:
import pandas as pd
import tensorflow as tf


from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [2]:
df=pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
###Drop Nan Values
df=df.dropna()
df.shape

(18285, 5)

In [5]:
X=df.drop('label',axis=1)
y=df['label']

In [6]:
### Vocabulary size
voc_size=5000

# Onehot Representation

In [7]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [8]:
messages.reset_index(inplace=True)

In [9]:
import nltk
import re
from nltk.corpus import stopwords

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [12]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [13]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[810, 1502, 3336, 468, 3970, 4410, 4778, 1555, 939, 277],
 [2346, 4487, 3238, 1320, 863, 160, 973],
 [3165, 1941, 4625, 2958],
 [67, 2451, 1179, 4416, 1283, 1626],
 [1592, 863, 4444, 3221, 1067, 429, 863, 4295, 665, 4484],
 [4749,
  4027,
  1768,
  3692,
  2525,
  2605,
  370,
  3511,
  2369,
  4859,
  493,
  4802,
  926,
  3813,
  973],
 [14, 1184, 3480, 2952, 3521, 318, 3271, 3713, 4005, 2579, 3846],
 [2493, 536, 1587, 2109, 4764, 1243, 2605, 1652, 4005, 2579, 3846],
 [1624, 4366, 795, 1984, 4061, 1535, 3550, 506, 2605, 175],
 [515, 4610, 3657, 4959, 4081, 2459, 1193, 448],
 [1335, 2711, 1272, 89, 839, 2229, 95, 3721, 1029, 3437, 1419],
 [4416, 4144, 3970, 1535, 2605, 4764],
 [4779, 4016, 4207, 1159, 3043, 1559, 663, 2529, 1957],
 [3464, 3321, 3225, 162, 3777, 3313, 2337, 4005, 2579, 3846],
 [4106, 3091, 4332, 3765, 3878, 4005, 2579, 3846],
 [1707, 1586, 1440, 2089, 1480, 1869, 2819, 2740, 762, 1142],
 [2207, 1730, 4487],
 [3861, 4919, 1432, 3833, 2605, 342, 4515, 973],
 [1887, 607,

# Embedding Representation

In [14]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1555  939  277]
 [   0    0    0 ...  863  160  973]
 [   0    0    0 ... 1941 4625 2958]
 ...
 [   0    0    0 ... 4005 2579 3846]
 [   0    0    0 ... 2536 3735 3193]
 [   0    0    0 ...  477 4796  888]]


In [15]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  810,
       1502, 3336,  468, 3970, 4410, 4778, 1555,  939,  277])

In [16]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
len(embedded_docs),y.shape

(18285, (18285,))

In [18]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [19]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Training

In [21]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 16s 81ms/step - loss: 0.3356 - accuracy: 0.8447 - val_loss: 0.1956 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 9s 47ms/step - loss: 0.1388 - accuracy: 0.9450 - val_loss: 0.1870 - val_accuracy: 0.9241
Epoch 3/10
192/192 [==============================] - 8s 40ms/step - loss: 0.0958 - accuracy: 0.9651 - val_loss: 0.2143 - val_accuracy: 0.9231
Epoch 4/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0653 - accuracy: 0.9770 - val_loss: 0.2423 - val_accuracy: 0.9188
Epoch 5/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0403 - accuracy: 0.9869 - val_loss: 0.3377 - val_accuracy: 0.9163
Epoch 6/10
192/192 [==============================] - 8s 42ms/step - loss: 0.0243 - accuracy: 0.9924 - val_loss: 0.3870 - val_accuracy: 0.9165
Epoch 7/10
192/192 [==============================] - 8s 43ms/step - loss: 0.0141 - accuracy: 0.9957 - val_loss: 0.4388 - val_accuracy: 0.914

# Adding Dropout

In [22]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Performance Metrics And Accuracy

In [23]:
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
confusion_matrix(y_test,y_pred)

array([[ 705, 2714],
       [ 722, 1894]], dtype=int64)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.4306545153272577